In [3]:
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
import re

In [4]:
df = pd.read_csv("../data/bundestag_wp20_speeches.csv")

# Display the first few rows
print(df.head())

           id                 title  \
0  2021400100        Johannes Vogel   
1  2021400200  Dr. Johannes Fechner   
2  2021400300     Dr. Bernd Baumann   
3  2021400400         Thorsten Frei   
4  2021400500     Dr. Irene Mihalic   

                                                text        date  protocol_id  \
0  Frau Präsidentin! Liebe Kolleginnen und Kolleg...  2025-03-18         5701   
1  Frau Präsidentin! Liebe Kolleginnen und Kolleg...  2025-03-18         5701   
2  Frau Präsidentin! Meine Damen und Herren! Eine...  2025-03-18         5701   
3  Sehr geehrte Frau Präsidentin! Liebe Kolleginn...  2025-03-18         5701   
4  Sehr geehrte Frau Präsidentin! Liebe Kolleginn...  2025-03-18         5701   

  protocol_number page_start  page_end topics  speaker_id  ...  \
0          20/214        NaN       NaN    NaN    11004179  ...   
1          20/214        NaN       NaN    NaN    11004270  ...   
2          20/214        NaN       NaN    NaN    11004662  ...   
3          20/

In [5]:
# ToDo
#  check if all procedural remarks are deleted
#  check if we want to add zwischenrufe
#  research question 3

In [6]:
# Splitting of the date column into days/weeks/months/year
df["date"] = pd.to_datetime(df["date"], format="ISO8601")
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day

df = df.sort_values(by="date")


In [7]:
df.head()

,id,title,text,date,protocol_id,protocol_number,page_start,page_end,topics,speaker_id,...,speaker_role,speaker_function,speaker_ministry,speaker_state,is_interjection,is_president,page_section,year,month,day
26066,20101400,Petra Pau,"Frau Präsidentin, ich nehme die Wahl an und fr...",2021-10-26,5432,20/1,NaN,NaN,NaN,11003206,...,NaN,NaN,NaN,NaN,False,False,NaN,2021,10,26
26053,20100100,Gabriele Katzmarek,Sehr geehrter Herr Alterspräsident! So muss ic...,2021-10-26,5432,20/1,NaN,NaN,NaN,11004325,...,NaN,NaN,NaN,NaN,False,False,NaN,2021,10,26
26054,20100200,Stefan Müller,Herr Präsident! Liebe Kolleginnen und Kollegen...,2021-10-26,5432,20/1,NaN,NaN,NaN,11003597,...,NaN,NaN,NaN,NaN,False,False,NaN,2021,10,26
26055,20100300,Britta Haßelmann,Sehr geehrter Herr Alterspräsident Wolfgang Sc...,2021-10-26,5432,20/1,NaN,NaN,NaN,11003764,...,NaN,NaN,NaN,NaN,False,False,NaN,2021,10,26
26056,20100400,Dr. Marco Buschmann,Sehr geehrter Herr Präsident! Meine lieben Kol...,2021-10-26,5432,20/1,NaN,NaN,NaN,11004023,...,NaN,NaN,NaN,NaN,False,False,NaN,2021,10,26


In [8]:
df["text"].str.lower().str.strip().str[:50].value_counts().head(50)

text
sehr geehrte frau präsidentin! liebe kolleginnen u    3610
frau präsidentin! liebe kolleginnen und kollegen!     1842
sehr geehrte frau präsidentin! sehr geehrte damen      917
sehr geehrter herr präsident! liebe kolleginnen un     736
sehr geehrte frau präsidentin! sehr geehrte kolleg     640
sehr geehrte frau präsidentin! meine damen und her     487
frau präsidentin! meine sehr verehrten damen und h     387
sehr geehrte frau präsidentin! meine sehr geehrten     379
sehr geehrte frau präsidentin! werte kolleginnen u     365
frau präsidentin! meine sehr geehrten damen und he     309
sehr geehrte frau präsidentin! sehr geehrter herr      258
sehr geehrte frau präsidentin! kolleginnen und kol     232
sehr geehrter herr präsident! sehr geehrte damen u     193
sehr geehrte frau präsidentin! meine sehr verehrte     189
sehr geehrte frau präsidentin! verehrte kolleginne     168
vielen dank. – frau präsidentin! liebe kolleginnen     151
frau präsidentin! sehr geehrte kolleginnen und kol 

In [9]:
# optimized_pattern = re.compile(
#     r"^(sehr geehrte(?:r)? (?:frau|herr)?(?: präsident(?:in)?)?"  # Matches "Sehr geehrte(r) Frau/Herr Präsident(in)"
#     r"[,!\s]*"  # Allows optional punctuation and spaces
#     r"(?:meine )?(?:sehr )?(?:verehrten|verehrte|geschätzten|geehrte|geehrten|liebe)? ?"  # Allows "meine", "sehr", and other honorifics
#     r"(?:damen und herren|kolleginnen und kollegen)?"  # Covers "Damen und Herren" or "Kolleginnen und Kollegen"
#     r"[,!\s]*"
#     r"(?:liebe|sehr geehrte|geschätzte)? ?(?:kolleginnen und kollegen|damen und herren)?"  # Ensures "liebe Kolleginnen und Kollegen" works
#     r"(?:[,!\s]*(?:der demokratischen fraktionen)?)?"  # Allows optional mention of "der demokratischen Fraktionen"
#     r")",
#     re.IGNORECASE | re.MULTILINE
# )


In [10]:
optimized_pattern = re.compile(
    r"^((?:vielen|herzlichen) dank(?:. – |, ))?" # catch beginning of "vielen/herzlichen dank - "
    r"(((sehr (?:geehrte|verehrte)(?:r)? )?"
    r"(?:frau|herr|glück auf|moin,)?( )?(?:(bundestags|alters)?präsident(?:in)?)?)?" # Catch any combination of sehr geehrte/verehrte Frau (Bundestags)Präsidentin
    r"[,!\s]*"
    r"(((sehr (?:geehrte|verehrte)(?:r)? )" # Catch any greetings to the kanzler/minister/regierung
    r"?(?:frau|herr)?(?: (bundes)kanzler|(bundes)minister|(bundes)regierung(?:in)?)?)?)?"
    r"[,!\s]*"
    r"(?:meine\s+)?(?:sehr\s+)?"
    r"(?:(verehrte|geschätzte|geehrte|liebe|werte)(?:n|r|s)?)?"
    r"\s*"
    r"(und)?"
    r"(?:damen\s+und\s+herren|(kolleginnen\s+und\s+)?kollegen)?" # catch damen und herren/kolleginnen und kollegen
    r"(abgeordnete)?"
    r"[,!\s]*"
    r"(?:(liebe|sehr\s+geehrte|geschätzte|werte)(?:n|r|s)?)?\s*(und)?"
    r"(?:kolleginnen\s+und\s+kollegen|damen\s+und\s+herren|publikum|präsidium|bürger(innen)|besucher(innen)\s+und\s+besucher)?" # catch damen und herren/kolleginnen und kollegen and other common persons/groups
    r"(auf den tribünen)?"
    r"(?:[,!\s]*(?:der\s+demokratischen\s+fraktionen)?)?)*"
    ,
    re.IGNORECASE | re.MULTILINE
)


In [11]:
# optimized_pattern = re.compile(
#     r"^(vielen dank. – )?(((sehr (?:geehrte|verehrte)(?:r)? )?(?:frau|herr)?(?: präsident(?:in)?)?)?[,!\s]*(?:meine\s+)?(?:sehr\s+)?(?:(verehrte|geschätzte|geehrte|liebe)(?:n))?\s*(?:damen\s+und\s+herren|kolleginnen\s+und\s+kollegen)?[,!\s]*(?:liebe|sehr\s+geehrte|geschätzte)?\s*(?:kolleginnen\s+und\s+kollegen|damen\s+und\s+herren)?(?:[,!\s]*(?:der\s+demokratischen\s+fraktionen)?)?)")


In [12]:
optimized_pattern

re.compile(r'^((?:vielen|herzlichen) dank(?:.\xa0– |, ))?(((sehr (?:geehrte|verehrte)(?:r)? )?(?:frau|herr|glück auf|moin,)?( )?(?:(bundestags|alters)?präsident(?:in)?)?)?[,!\s]*(((sehr (?:geehrte|verehrte)(?:r)? )?(?:frau|herr)?(?: (bundes)kanzler|(bundes)minister|(bundes)regierung(?:in)?)?)?)?[,!\s]*(?:meine\s+)?(?:sehr\s+)?(?:(verehrte|geschätzte|geehrte|liebe|werte)(?:n|r|s)?)?\s*(und)?(?:damen\s+und\s+herren|(kolleginnen\s+und\s+)?kollegen)?(abgeordnete)?[,!\s]*(?:(liebe|sehr\s+geehrte|geschätzte|werte)(?:n|r|s)?)?\s*(und)?(?:kolleginnen\s+und\s+kollegen|damen\s+und\s+herren|publikum|präsidium|bürger(innen)|besucher(innen)\s+und\s+besucher)?(auf den tribünen)?(?:[,!\s]*(?:der\s+demokratischen\s+fraktionen)?)?)*',
           re.IGNORECASE|re.MULTILINE|re.UNICODE)

In [13]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove punctuation and digits
    #text = re.sub(r'\W+', ' ', text)
    #text = re.sub(r'\d+', '', text)

    text = text.strip()

    text = re.sub(optimized_pattern, "", text).strip()  # Remove pattern if at the start

    return text

In [14]:
df["processed_text"] = df["text"].astype(str).apply(preprocess_text)

In [15]:
df["processed_text"].str.lower().str[:50].value_counts().head(50)

processed_text
s: stellungnahme gemäß artikel 76 \nabsatz 2 gg  .     23
bürger! frau präsidentin! meine damen und herren!      20
sordnung  . . . . . . . . . . . . . . . . . . . .      17
wehrbeauftragte! liebe kolleginnen und kollegen! m     13
ministerin! liebe kolleginnen und kollegen! liebe      12
kolleginnen! liebe kollegen! liebe bürgerinnen und     11
kolleginnen! liebe kollegen! sehr geehrte damen un     11
staatssekretärin! liebe kolleginnen und kollegen!      10
guten morgen, frau präsidentin! liebe kolleginnen      10
ng: überweisung an die zuständigen \nausschüsse  .     10
ministerin! liebe kolleginnen und kollegen! meine      10
kolleginnen! liebe kollegen! meine sehr geehrten d     10
ich schwöre es, so wahr mir gott helfe.\nherzlichen     8
grüß gott, frau präsidentin! meine sehr verehrten       8
landsleute! und natürlich: werte leugner des natür      8
der demokratischen parteien! sehr geehrte damen un      8
ministerin! sehr geehrte kolleginnen und kollegen!      7

In [16]:
df[["processed_text", "text", "speaker_party"]]

,processed_text,text,speaker_party
26066,ich nehme die wahl an und freue mich auf die z...,"Frau Präsidentin, ich nehme die Wahl an und fr...",DIE LINKE
26053,"so muss ich es jetzt sagen, genau; ich muss mi...",Sehr geehrter Herr Alterspräsident! So muss ic...,SPD
26054,die konstituierung des bundestages ist ein fes...,Herr Präsident! Liebe Kolleginnen und Kollegen...,CDU/CSU
26055,wolfgang schäuble! sehr geehrte kolleginnen un...,Sehr geehrter Herr Alterspräsident Wolfgang Sc...,BÜNDNIS 90/DIE GRÜNEN
26056,als ich gestern insbesondere die neuen kollegi...,Sehr geehrter Herr Präsident! Meine lieben Kol...,FDP
...,...,...,...
26,unser alltag im bundestag ist ja oft sehr hekt...,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...,BÜNDNIS 90/DIE GRÜNEN
25,die gute nachricht vorneweg: eine dreistündige...,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,CDU/CSU
24,es wird in dieser so ernsten und wichtigen ang...,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...,SPD
34,– –\n(die abgeordneten des bsw halten transpar...,Sehr geehrte Frau Präsidentin! Werte – –\n\n(D...,SPD


In [ ]:
# Load the German spaCy model
nlp = spacy.load("de_core_news_sm")

# Download German stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("german"))

def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove punctuation and digits
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)


    text = re.sub(r"\(.*?\)", "", text)

    return " ".join(text)

In [22]:
intro_patterns = [
    r"sehr geehrte frau präsidentin! meine sehr geehrten damen und herren! liebe kolleginnen und kollegen!",
    r"sehr geehrte frau präsidentin! liebe kolleginnen und kollegen! meine sehr geehrten damen und herren!",
    r"sehr geehrte frau präsidentin! sehr geehrte kolleginnen und kollegen! sehr geehrte damen und herren!",
    r"sehr geehrte frau präsidentin! sehr geehrte kolleginnen und kollegen der demokratischen fraktionen!"
    r"sehr geehrte damen und herren",
    r"herr präsident[,]? meine damen und herren",
    r"frau präsidentin[,]? verehrte kolleginnen und kollegen",
    r"liebe kolleginnen und kollegen"
]

In [23]:
# Load the German spaCy model
nlp = spacy.load("de_core_news_sm")

# Download German stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("german"))

def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove punctuation and digits
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)

    # Remove intro patterns
    for pattern in intro_patterns:
        text = re.sub(rf"^{pattern}[:,]*\s*", "", text)  # Remove pattern if at the start

    # Tokenization and Lemmatization
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct and not token.is_space]

    return " ".join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
custom_stopwords = {"sagen", "gehen", "geben", "kommen", "müssen", "vieler", "brauchen", "sein", "mal", "ganz", "sagen", "schon", "tun"}


# To Discuss: mehr/weniger, gut/schlecht, ja/nein, wichtig, heute,

In [29]:
def remove_custom_stopwords(text):
    """Removes custom stopwords from a lemmatized text."""
    tokens = text.split()  # Split text into words
    filtered_tokens = [word for word in tokens if word not in custom_stopwords]
    return " ".join(filtered_tokens)

In [26]:
stop_words

{'aber',
 'alle',
 'allem',
 'allen',
 'aller',
 'alles',
 'als',
 'also',
 'am',
 'an',
 'ander',
 'andere',
 'anderem',
 'anderen',
 'anderer',
 'anderes',
 'anderm',
 'andern',
 'anderr',
 'anders',
 'auch',
 'auf',
 'aus',
 'bei',
 'bin',
 'bis',
 'bist',
 'da',
 'damit',
 'dann',
 'das',
 'dass',
 'dasselbe',
 'dazu',
 'daß',
 'dein',
 'deine',
 'deinem',
 'deinen',
 'deiner',
 'deines',
 'dem',
 'demselben',
 'den',
 'denn',
 'denselben',
 'der',
 'derer',
 'derselbe',
 'derselben',
 'des',
 'desselben',
 'dessen',
 'dich',
 'die',
 'dies',
 'diese',
 'dieselbe',
 'dieselben',
 'diesem',
 'diesen',
 'dieser',
 'dieses',
 'dir',
 'doch',
 'dort',
 'du',
 'durch',
 'ein',
 'eine',
 'einem',
 'einen',
 'einer',
 'eines',
 'einig',
 'einige',
 'einigem',
 'einigen',
 'einiger',
 'einiges',
 'einmal',
 'er',
 'es',
 'etwas',
 'euch',
 'euer',
 'eure',
 'eurem',
 'euren',
 'eurer',
 'eures',
 'für',
 'gegen',
 'gewesen',
 'hab',
 'habe',
 'haben',
 'hat',
 'hatte',
 'hatten',
 'hier',


In [27]:
df["processed_text"] = df["processed_text"].astype(str).apply(preprocess_text)

TypeError: remove_custom_stopwords() missing 2 required positional arguments: 'text' and 'stopword_list'

In [30]:
df["processed_text"] = df["processed_text"].astype(str).apply(remove_custom_stopwords)

df["is_president"] = df["is_president"].astype(bool)  # Ensure it's Boolean (True/False)

# Show processed text
print(df["processed_text"].head())

26225         nehmen Wahl freuen Zusammenarbeit Demokratie
26212    genau daran gewöhnen lange lange recht lieb Ko...
26213    Konstituierung Bundestag Festtag Demokratie We...
26214    Wolfgang Schäuble geehrt kollegin Kollege Tat ...
26215    gestern insbesondere neu kolleginn Kollege Fra...
Name: processed_text, dtype: object


In [31]:
# Splitting of the date column into days/weeks/months/year
df["date_fixed"] = pd.to_datetime(df["date"], format="ISO8601")
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day


In [32]:
def clean_party_name(party):
    if pd.isna(party):
        return "Unknown"
    party = party.strip()  # Remove leading/trailing spaces
    party = party.replace("\xa0", " ")  # Replace non-breaking spaces with normal spaces
    party = " ".join(party.split())  # Remove extra spaces and newlines
    party = party.replace("\n", " ")  # Replace newlines with spaces
    party = party.replace("DIE LINKE", "Die Linke")  # Standardize "Die Linke"
    party = party.replace("BÜNDNIS 90/DIE GRÜNEN", "Bündnis 90/Die Grünen")  # Standardize
    party = party.replace("Fraktionslos", "fraktionslos")
    party = party.replace("SPDCDU/CSU", "Unknown")  # Fix odd merged entry
    return party

# Apply the cleaning function
df["party_clean"] = df["speaker_party"].astype(str).apply(clean_party_name)

# Get unique cleaned parties
print("Cleaned Parties:", df["party_clean"].unique())

Cleaned Parties: ['Die Linke' 'SPD' 'CDU/CSU' 'Bündnis 90/Die Grünen' 'FDP' 'AfD' 'nan'
 'fraktionslos' 'Unknown' 'BSW']


In [33]:
df.to_csv("../data/bundestag_wp20_speeches_preprocessed.csv")